In [1]:
!pip install transformers


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
!pip install transformers
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_data = pd.read_csv("/content/drive/MyDrive/yelp_review_full_csv/augmented_train_6500.csv",header = None)
test_data = pd.read_csv("/content/drive/MyDrive/yelp_review_full_csv/test.csv",header = None)

#train_data = train_data.iloc[: , 1:]
train_data.columns = [2 , 0 , 1]
train_data[0][0] = 1
#train_data = train_data.iloc[1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_data.shape

(19471, 3)

In [6]:
train_data.head()

,2,0,1
0,NaN,1,1
1,0.0,5,dr goldberg offers everything i look for in a ...
2,1.0,5,dr goldberg offers everything see look for in ...
3,2.0,5,dr goldberg offers everything i look for in a ...
4,3.0,2,unfortunately the frustration of being dr gold...


In [7]:
max(train_data[1].apply(len))

5646

In [8]:
#x_,x_train,y_,y_train = train_test_split(train_data[1],train_data[0],test_size=0.01)

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

model, pretrained_weights = (transformers.DistilBertModel, 'distilbert-base-uncased')
model = model.from_pretrained(pretrained_weights)

#for child in model.children():
#  for param in child.parameters():
#      param.requires_grad = False

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
print(' Original: ', train_data[1][1])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train_data[1][1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train_data[1][1])))

 Original:  dr goldberg offers everything i look for in a general practitioner hes nice and easy to talk to without being patronizing hes always decent on time in seeing his patients hes affiliated with a top notch hospital very along nyu which my parents angstrom have explained to me is very important in case something happens check and you need surgery and you can get pose referrals to see specialists without having to see him first really infirmary what more do you need im sitting here trying to think of more than any complaints i have about only him but im really drawing a blank
Tokenized:  ['dr', 'goldberg', 'offers', 'everything', 'i', 'look', 'for', 'in', 'a', 'general', 'practitioner', 'he', '##s', 'nice', 'and', 'easy', 'to', 'talk', 'to', 'without', 'being', 'patron', '##izing', 'he', '##s', 'always', 'decent', 'on', 'time', 'in', 'seeing', 'his', 'patients', 'he', '##s', 'affiliated', 'with', 'a', 'top', 'notch', 'hospital', 'very', 'along', 'nyu', 'which', 'my', 'parents', 

In [11]:
#x_token = x_train.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)[:300]))

#padded = np.array([i + [0]*(300 -len(i)) for i in x_token.values])

In [12]:
#attention_mask = np.where(padded != 0, 1, 0)


In [13]:
class Yelp_Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        review = str(self.data[1][index])
        inputs = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation = True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data[0][index], dtype=torch.long)
        }
        
    def __len__(self):
        return len(self.data)
    
train_dataset = Yelp_Dataset(train_data, tokenizer,200)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,drop_last = True)

test_dataset = Yelp_Dataset(test_data[0:5000], tokenizer,200)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True,drop_last = True)

In [14]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.distill_bert = model
        self.drop = torch.nn.Dropout(0.1)
        self.out = torch.nn.Linear(768, 5)
    
    def forward(self, ids, mask):
        distilbert_output = self.distill_bert(ids, mask)
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        output_1 = self.drop(pooled_output)
        output = self.out(output_1)
        return output
    
model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (distill_bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Line

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [16]:
def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct


In [17]:

for epoch in range(3):
    acc = 0
    for i,data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        labels = data['targets'].to(device)
        labels = labels - 1
        outputs = model(input_ids,attention_mask)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        acc += accuracy(outputs,labels)
        if i%200 == 199:
            print('[%d, %d] accuracy: %.3f' %
                  (epoch + 1, i + 1, acc / ((i+1)*16)))





/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[1, 200] accuracy: 0.227
[1, 400] accuracy: 0.284
[1, 600] accuracy: 0.343
[1, 800] accuracy: 0.378
[1, 1000] accuracy: 0.407
[1, 1200] accuracy: 0.425
[2, 200] accuracy: 0.536
[2, 400] accuracy: 0.536
[2, 600] accuracy: 0.542
[2, 800] accuracy: 0.546
[2, 1000] accuracy: 0.553
[2, 1200] accuracy: 0.559
[3, 200] accuracy: 0.583
[3, 400] accuracy: 0.591
[3, 600] accuracy: 0.593
[3, 800] accuracy: 0.600
[3, 1000] accuracy: 0.605
[3, 1200] accuracy: 0.606


In [18]:

acc = 0
with torch.no_grad():
    for i,data in enumerate(test_loader,0):
        model.eval()
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        labels = data['targets'].to(device)
        labels = labels - 1
        outputs = model(input_ids,attention_mask)
        acc += accuracy(outputs,labels)
print(i)
acc/((i+1)*16)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


311


0.5614983974358975